# Scientific Programming in Python: Tabular Data with `pandas`

This tutorial focuses on introducing the libraries [`numpy`](https://numpy.org/doc/stable/) and [`pandas`](https://pandas.pydata.org/docs/). It additionally uses [`matplotlib`](https://matplotlib.org/) and [`seaborn`](https://seaborn.pydata.org/index.html) for visualization. The goal is to get the learner familiar with the basic operations of each library and their general capabilities as well as some common paradigms in how they are used.

This notebook contains the tutorial on `pandas`.

For starters, we will need to import each of these libraries. Canonically, these libraries have common import aliases.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

## Tabular Data

A lot of scientific data is easily expressed, stored, and manipulated as tabular data. Tabular data is typically any data that can be represented in a spreadsheet or table. In Pandas, these tables are called dataframes. They are in many ways similar to `numpy` matrices, but they allow one to index rows and columns by names or specific numeric identifiers rather than natural numbers. Pandas also supports many utilities for combining and merging dataframes.

In [ ]:
from cloudpathlib import S3Path, S3Client

client = S3Client(
    no_sign_request=True,
    local_cache_dir='/home/jovyan/cache')
nsd_root = S3Path('s3://natural-scenes-dataset/', client=client)

behav_dirpath = nsd_root / 'nsddata' / 'ppdata' / 'subj01' / 'behav'
with (behav_dirpath / 'responses.tsv').open('r') as file:
    behav_data = pd.read_csv(file, sep='\t')

behav_data

## Basic Operations

Pandas has a syntax that generally resembles that of NumPy matrices, but that enables rows and columns to have names or indices other than the natural numbers (as all arrays use). For example, a common operation on a pandas dataframe is to select the rows of the dataframe that have a particular property, such as selecting just the data from session 1 of the NSD experiments.

In [ ]:
ses1_data = behav_data[behav_data['SESSION'] == 1]
ses1_data

Similarly, we can select the rows from session 1 that are part of run 1.

In [ ]:
run1_data = ses1_data[ses1_data['RUN'] == 1]
run1_data

Alternatively, we could have selected all of the session 1 run 1 rows in one go if we combine the conditions using a logical intersection `&` operator:

In [ ]:
behav_data[(behav_data['SESSION'] == 1) & (behav_data['RUN'] == 1)]

Now that we have selected a single subject, session, and run, we might want to look at the distribution of reaction times (the `'RT'` column) that the subject had when doing the task. We can access columns of a datafram as if the dataframe were a dictionary of columns.

In [ ]:
plt.hist(run1_data['RT'])

Notice that there are multiple meanings to the syntax `dataframe[arg]` depending on the `arg`. If `arg` is a column name (string) or a list of column names, then those columns are selected and dataframe representing only the listed column(s) is returned. If `arg` is a boolean mask, then it is applied to the rows.

To access sub-elements or sub-dataframes of a dataframe according to its axes (which are the same as in a NumPy array: axis 0 is the rows and axis 1 is the columns), one can use either the `dataframe.loc` or `dataframe.iloc` interfaces.

The `dataframe.iloc[rows, cols]` interface is basically equivalent to indexing into a numpy matrix; row `0` will be the first row in the dataframe and column `0` will be the first column.

The `dataframe.loc[rows, cols]`, on the other hand, expects the `rows` and `cols` to be index values in the dataframe. The dataframes we've loaded and made so far have been automatically given row indices by Pandas that are equivalent to the NumPy array indices (i.e., the first row has index 0, the next has index 1, etc.). The columns are indexed according to the column names, however.

In [ ]:
# Extract the first 10 rows of the RT column using iloc; the column index for
# 'RT' is 9:
run1_data.iloc[:10, 9]

In [ ]:
# Extract the first 10 rows of the RT column using loc:
run1_data.loc[:10, 'RT']

Since the rows of run1_data are naturally indexed by the trial number, we might want to give this dataframe its own index. In this case, it would mean that `run1_data.loc[1]` returned the row for trial 1; currently `run1_data.loc[0]` is required to extract trial 1.

Because Python uses zero-based indexing, one may or may not prefer to make this indexing change, but in many cases, for example in projects like the Human Connectome Project where subjects have random IDs like `111312`, it can be useful to request `dataframe.loc[111312]` instead of figuring out which row contains that particular subject.

In [ ]:
run1_data_indexed = run1_data.set_index('TRIAL')
run1_data_indexed.loc[[1,2,3]]

In [ ]:
run1_data_indexed.iloc[[1,2,3]]

Certain pandas operations return dataframes whose rows *and* columns are labeled by strings instead of numbers. Rows can have many kinds of indices, and it's possible to make dataframes with various kinds of row indices including multi-indices (see the [pandas guide to indexing and selecting data](https://pandas.pydata.org/docs/user_guide/indexing.html) for more information).

The `corr` method is an example of a method that produces a dataframe whose rows and columns are labeled by strings.

In [ ]:
# Select a subset of the columns that we care about.
data = behav_data[['SUBJECT', 'SESSION', 'RUN', 'RT', 'ISCORRECT']]

# Calculate the correlation of all columns to all others; use the
# Spearman (rank) correlation.
corr = data.corr(method='spearman')

# Show the resulting dataframe:
corr

To select rows and columns, we can use `loc` and `iloc` just as before, but when using `loc`, the row values are strings now.

In [ ]:
corr.loc[['SESSION', 'RUN'], 'RT']

## Grouping DataFrames

One of the most common operations one needs to run on a dataframe is to subdivide it into smaller dataframes then to perform operations over those subsets of the dataframe, for example to run an analysis over each individual session in our subject's behavioral dataframe.

This can be done by looping over a dataframe and making the clusters manually, but there are builtin methods that can make this a lot easier. Let's look at how we would divide the session 1 data from our subject up into runs then calculate some basic statistics on each run.

In [ ]:
# We use the groupby method; here we provide just one column name, but a list
# of columns can also be given.
ses1_groups = ses1_data.groupby('RUN')

# The groupby method returns a special kind of group-by object:
ses1_groups

In [ ]:
# If we want to calculate the mean of the columns in each group, there's a
# builtin method for that! It even returns a dataframe indexed by the column
# (or columns) that we grouped by:
means = ses1_groups.mean()

# Notice that the index of the returned dataframe is the RUN column:
means

In [ ]:
# Similarly we can calculate other statistics like the standard deviation:
stds = ses1_groups.std()

In [ ]:
# One can also loop over a group:
for (run_id, group) in ses1_groups:
    print(f'Run {run_id:2d}: mean RT = {np.mean(group["RT"]):7.2f}')

## Combining DataFrames

Now that we've created a dataframe of the means of the columns for each run and the standard deviations of the columns for those runs in the section above (`means` and `stds`), we might want to examing how the reaction time changes from one run to the next. We might even hypothesize that as the subject practices the task, they will get faster.

To do this analysis, we might want to start by reducing the dataframes down to just the parts we care about (e.g., the reaction times `'RT'` and whether the got the trial correct `'ISCORRECT'`).

In [ ]:
means = means[['ISCORRECT', 'RT']]
stds = stds[['ISCORRECT', 'RT']]

# Look at the means:
means

### Merging DataFrames

It would be nice for the sake of other operations we might want to do, like iterating through the data or plotting, if we had the columns of the `means` dataframe and the columns of the `stds` dataframe together in a single dataframe, for example with the columns `ISCORRECT_mean`, `ISCORRECT_std`, `RT_mean`, and `RT_std`.

To combine dataframes like this, we use the `pandas.merge` function.

In [ ]:
# Merge the means and stds together using the column RUN to decide whether two
# rows of the dataframe should be merged.
rt_stats = pd.merge(means, stds, on='RUN')
rt_stats

The above cell worked fine for combining the columns, but we'd like the columns to have specific names that are meaningful to us:

In [ ]:
rt_stats = pd.merge(means, stds, on='RUN', suffixes=('_mean', '_std'))
rt_stats

In [ ]:
# Notice that the RUN number in these dataframes is now part of the index, and
# so we access it using .index instead of ['RUN']:
rt_stats.index

To test our theory about reaction times, we can extract data by column to make a quick matplotlib plot of the average reaction time across trials, ± the standard deviation.

In [ ]:
plt.errorbar(
    rt_stats.index, rt_stats['RT_mean'], rt_stats['RT_std'],
    fmt='o:')

This plot doesn't really support our hypothesis that the reaction times would decrease across runs as the subject gained experience.

A better plot would use the standard error instead of the standard deviation. To obtain this, we would need to get the count of trials in each of our groups. This can be accomplished with the `count` method.

In [ ]:
# Produce a dataframe similar to means and stds but containing counts of the
# number of trials in each run.
counts = ses1_groups.count()[['ISCORRECT', 'RT']]

# Let's try merging without suffixes...
pd.merge(rt_stats, counts, on='RUN')

In [ ]:
# To get nice columns like RT_count into our dataframe, we can rename the
# columns of counts to RT_count and ISCORRECT_count then merge on RUN:
rename_cols = {'ISCORRECT': 'ISCORRECT_count', 'RT': 'RT_count'}
rt_stats = pd.merge(
    rt_stats,
    counts.rename(columns=rename_cols),
    on='RUN')

rt_stats

In [ ]:
# Now we can remake our error plot using standard error.
plt.errorbar(
    rt_stats.index, rt_stats['RT_mean'],
    rt_stats['RT_std'] / np.sqrt(rt_stats['RT_count']),
    fmt='o:')
plt.xlabel('Run Number')
plt.ylabel('Reaction Time [ms]')

### Concatenating DataFrames

Another common way that we need to combine dataframes is when we have several dataframes with the same columns that need to be appended together. For this we can use the `pandas.concat` function. This function requires a list of dataframes which are concatenated vertically in order.

For example, let's load in the NSD behavioral data for all of the subjects and concatenate them into one large dataframe that contains all subjects, sessions, runs, and trials!

In [ ]:
# This is the data that contains preprocessed subject data; each subejct has a
# sub-directory in this directory.
ppdata_dirpath = nsd_root / 'nsddata' / 'ppdata'

# We'll iterate through each subdirectory (one per subject), load in their
# dataframe, and append it to the all_data list (which we will pass into
# pd.concat at the end of the loop.
all_data = []
for subj_dirpath in ppdata_dirpath.iterdir():
    with (subj_dirpath / 'behav' / 'responses.tsv').open('r') as file:
        subj_behav_data = pd.read_csv(file, sep='\t')
    all_data.append(subj_behav_data)

# Concatenate all the individual dataframes into one giant one:
all_data = pd.concat(all_data, ignore_index=True)

In [ ]:
all_data

## Seaborn For Visualizing DataFrames

A very useful library for quickly looking at variation in a dataset that is stored in a pandas dataframe is a library called [Seaborn](https://seaborn.pydata.org/). Seaborn offers a fairly intuitive interface to many useful scientific plots, and learning its interface is often best done through examples: see in particular the [Seaborn example gallery](https://seaborn.pydata.org/examples/index.html).

For example, if we wanted to look at how the reaction times of subjects varied across runs in their first session, we could select the subdataframe for the first session then instruct Seaborn to organize a set of violin-plots of the reaction time by subject and run.

In [ ]:
# Seaborn is typically imported as sns.
import seaborn as sns

# Make a matplotlib figure and one set of axes on which to draw.
(fig,ax) = plt.subplots(1,1, figsize=(7,5), dpi=288)

# We use seaborn's violinplot function:
sns.violinplot(
    # The data we want seaborn to plot:
    data=all_data[all_data['SESSION'] == 1],
    # Along the x-axis we want histograms grouped by subject.
    x='SUBJECT',
    # We want the histograms to be the reaction time, along the y-axis.
    y='RT',
    # We want to additionally color the different histograms by the run.
    hue='RUN',
    # We don't want to draw boundary lines on the histograms.
    linewidth=0,
    # Plot on this set of axes:
    ax=ax)

plt.show()